# 02 - Data Cleaning

Clean the LendingClub dataset for modeling.

**Steps**:
1. Filter to binary outcomes (Fully Paid / Charged Off)
2. Create target variable
3. Handle missing values
4. Remove data leakage columns

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Paths
RAW_DATA = Path("../data/raw/lending_club_raw.parquet")
INTERIM_DATA = Path("../data/interim")
INTERIM_DATA.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load raw data
df = pd.read_parquet(RAW_DATA)
print(f"Loaded data shape: {df.shape}")

## 1. Filter to Binary Outcomes

Keep only loans with clear outcomes:
- **Fully Paid**: Loan successfully repaid
- **Charged Off**: Loan defaulted (borrower stopped paying)

In [ ]:
# Current distribution
print("Original loan status distribution:")
print(df['loan_status'].value_counts())

In [ ]:
# Filter to binary outcomes
target_statuses = ['Fully Paid', 'Charged Off']
df = df[df['loan_status'].isin(target_statuses)].copy()

print(f"\nAfter filtering: {df.shape[0]:,} rows")
print(df['loan_status'].value_counts())

## 2. Create Target Variable

In [ ]:
# Create binary target: 1 = Default (Charged Off), 0 = Fully Paid
df['default'] = (df['loan_status'] == 'Charged Off').astype(int)

print("Target variable distribution:")
print(df['default'].value_counts())
print(f"\nDefault rate: {df['default'].mean():.2%}")

## 3. Handle Missing Values

In [ ]:
# Identify columns with missing values
missing_counts = df.isnull().sum()
missing_pct = (missing_counts / len(df) * 100).sort_values(ascending=False)

print("Columns with >50% missing (will drop):")
high_missing = missing_pct[missing_pct > 50]
print(high_missing)

In [ ]:
# Drop columns with >50% missing
cols_to_drop = missing_pct[missing_pct > 50].index.tolist()
df = df.drop(columns=cols_to_drop)
print(f"Dropped {len(cols_to_drop)} columns with >50% missing values")
print(f"Remaining columns: {df.shape[1]}")

In [ ]:
# Handle specific missing values

# Employment length: fill with '0 years' (assume no employment history)
if 'emp_length' in df.columns:
    df['emp_length'] = df['emp_length'].fillna('0 years')
    # Extract numeric value
    df['emp_length_numeric'] = df['emp_length'].str.extract(r'(\d+)').astype(float)
    df['emp_length_numeric'] = df['emp_length_numeric'].fillna(0)
    print(f"emp_length_numeric: {df['emp_length_numeric'].describe()}")

In [ ]:
# Annual income: fill with median
if 'annual_inc' in df.columns:
    median_income = df['annual_inc'].median()
    missing_before = df['annual_inc'].isnull().sum()
    df['annual_inc'] = df['annual_inc'].fillna(median_income)
    print(f"annual_inc: filled {missing_before} missing with median ${median_income:,.0f}")

In [ ]:
# Revolving utilization: fill with median
if 'revol_util' in df.columns:
    median_util = df['revol_util'].median()
    missing_before = df['revol_util'].isnull().sum()
    df['revol_util'] = df['revol_util'].fillna(median_util)
    print(f"revol_util: filled {missing_before} missing with median {median_util:.1f}%")

In [ ]:
# Months since features: fill with -1 (indicates "never occurred")
mths_cols = [c for c in df.columns if 'mths_since' in c.lower()]
for col in mths_cols:
    missing_before = df[col].isnull().sum()
    if missing_before > 0:
        df[col] = df[col].fillna(-1)
        print(f"{col}: filled {missing_before} missing with -1")

In [ ]:
# DTI: fill with median
if 'dti' in df.columns:
    median_dti = df['dti'].median()
    missing_before = df['dti'].isnull().sum()
    df['dti'] = df['dti'].fillna(median_dti)
    print(f"dti: filled {missing_before} missing with median {median_dti:.1f}")

## 4. Remove Data Leakage Columns

**Critical**: Remove columns that contain information not available at loan origination.

These include:
- Payment history (when/how much was paid)
- Collection and recovery information
- Current loan status derived fields

In [ ]:
# Data leakage columns to remove
leakage_cols = [
    # Payment information (post-origination)
    'issue_d', 'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
    'next_pymnt_d',
    
    # Collection/recovery (post-default)
    'collection_recovery_fee', 'recoveries',
    
    # Total payment info (only known after loan ends)
    'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 
    'total_rec_late_fee', 'total_rec_prncp',
    
    # Outstanding principal (changes over time)
    'out_prncp', 'out_prncp_inv',
    
    # Funded amounts (could differ from requested)
    'funded_amnt', 'funded_amnt_inv',
    
    # Hardship/settlement (post-origination)
    'hardship_flag', 'debt_settlement_flag', 'debt_settlement_flag_date',
    'settlement_status', 'settlement_date', 'settlement_amount',
    'settlement_percentage', 'settlement_term',
    
    # Original loan status (we have 'default' now)
    'loan_status',
    
    # Other post-origination
    'pymnt_plan', 'hardship_type', 'hardship_reason', 'hardship_status',
    'hardship_amount', 'hardship_start_date', 'hardship_end_date',
    'hardship_length', 'hardship_dpd', 'hardship_loan_status',
    'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'orig_projected_additional_accrued_interest', 'payment_plan_start_date'
]

# Remove existing leakage columns
existing_leakage = [c for c in leakage_cols if c in df.columns]
df = df.drop(columns=existing_leakage, errors='ignore')
print(f"Removed {len(existing_leakage)} data leakage columns")
print(f"Remaining columns: {df.shape[1]}")

In [ ]:
# Also remove ID columns and URLs (not useful for modeling)
id_cols = ['id', 'member_id', 'url', 'desc', 'title', 'zip_code', 'addr_state', 'emp_title']
existing_id_cols = [c for c in id_cols if c in df.columns]
df = df.drop(columns=existing_id_cols, errors='ignore')
print(f"Removed {len(existing_id_cols)} ID/text columns")

## 5. Final Cleaning Check

In [ ]:
# Check remaining missing values
remaining_missing = df.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0].sort_values(ascending=False)
print(f"Columns still with missing values: {len(remaining_missing)}")
if len(remaining_missing) > 0:
    print(remaining_missing.head(20))

In [ ]:
# Fill remaining numeric columns with median
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

print(f"Filled remaining numeric missing values with median")

In [ ]:
# Final summary
print("\n" + "="*50)
print("CLEANING SUMMARY")
print("="*50)
print(f"Final shape: {df.shape}")
print(f"Target variable: 'default'")
print(f"Default rate: {df['default'].mean():.2%}")
print(f"Total missing values: {df.isnull().sum().sum()}")

In [ ]:
# List remaining columns
print("\nRemaining columns:")
for i, col in enumerate(df.columns):
    dtype = df[col].dtype
    print(f"{i+1:3}. {col:40} ({dtype})")

## 6. Save Cleaned Data

In [ ]:
# Save to interim folder
output_path = INTERIM_DATA / "lending_club_cleaned.parquet"
df.to_parquet(output_path, index=False)

import os
file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Saved to: {output_path}")
print(f"File size: {file_size_mb:.1f} MB")

## Next Steps

Proceed to `03_feature_engineering.ipynb` to:
- Create financial ratios (loan-to-income, payment burden)
- Encode categorical variables
- Prepare final feature set